### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.


In [1]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

## 에어비앤비 주요 지역 탐색 

### Inside AIrbne 주요 지역 기준 오시마랜드 데이터 수집 

InsideAirbnb의 geojosn 위치정보 파일 사용

In [2]:
import json
import pandas as pd

In [6]:
geo_df = pd.read_json('data/japan.geojson') # pd.read_json 이용
# listing_data = pd.read_csv('../../Tokyo_Airbnb/data/listings.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')


In [33]:
features = geo_df['features'][:15]

geo_dict = {}
for feature in features:
    region = feature['properties']['nam']
    coordinates = feature["geometry"]['coordinates'][0][0]
    for idx, lat_lng in enumerate(coordinates):
        geo_dict[f'{region}_{idx}'] = lat_lng

print(len(geo_dict))

8500


#### Osaka Station

In [61]:
osaka_station = pd.read_csv('data/osaka_station.csv')
osaka_station

,Ｓｔａｔｉｏｎ
0,Higashimikuni
1,Shin-Osaka
2,Nishinakajima-Minamigata
3,Nakatsu
4,Umeda
...,...
119,Ferry Terminal
120,Nanko-higashi
121,Nanko-guchi
122,Hirabayashi


## 크롤링 
- 크롤링 메서드 생성 

####  1. oshimaland 웹사이트 불러오기


In [89]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [90]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        #전체 창 
        
        map =  driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div')
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = []
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        original_window_handle = driver.current_window_handle  # 원래 창의 핸들을 저장


        print('fire 개수: ', len(fire_list))
        if len(fire_list) != 0 :
            for fire in fire_list:
                try: 
                    is_enabled = fire.is_enabled()
                    actions = ActionChains(driver)                 
                    if is_enabled:
                        # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                        # print('click OK')
                        if fire.is_displayed():
                    
                            actions = ActionChains(driver)                 
                            actions.move_to_element(fire).click().perform()

                            # time.sleep(5)
                            WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                            info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                            info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                            info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                            tokyo_data_dict.append({
                                'district': district,
                                'date': info_date,
                                'address': info_address,
                                'content': info_content
                            })

                            time.sleep(3)
                                    # 새로운 탭이 열렸는지 확인하고, 열린 경우 원래 창으로 다시 돌아옵니다.
                            if len(driver.window_handles) > 1:
                                driver.switch_to.window(original_window_handle)
                        else:
                            pass
                            # print('display NOT')
                    else:
                        pass
                        # print('click not')

                except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                    # 에러 처리
                    print(f"Error occurred: {e}")
                    map.send_keys(Keys.ENTER)

                    

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 지역 검색 


#### 도쿄 지역 위도 경도 검색

In [91]:
def search_fire_coordinate(driver, geo_dict = None):
    tokyo_data_dict = []
    # 주요 지역 위도 경도 검색 
    # for key, value in tqdm(geo_dict.items()): # geojson
    for value in osaka_station['Ｓｔａｔｉｏｎ'].values: # geojson
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        region_search_tab.clear()  # 검색어 clear
        # lng = value[1]
        # lat = value[0]
        # #경도 위도 검색
        # search  = f'{lng} {lat}'
        search = value
        region_search_tab.send_keys(search)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, search)



### 크롤링 

In [92]:
if __name__ == "__main__":
    # 드라이버 생성 및 웹 로딩
    driver = webdriver.Firefox()
    url = 'https://www.oshimaland.co.jp/'
    loadWeb(driver, url)

    # 지도를 60% 정도로 축소 뒤 수집 
    search_fire_coordinate(driver)
    

# driver.quit()

fire 개수:  41
Error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



ElementNotInteractableException: Message: Element <div class="mk-map-node-element"> is not reachable by keyboard
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:351:5
webdriverSendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:668:13
interaction.sendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:642:11
sendKeysToElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:535:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:150:31


#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

### 크롤링 데이터 저장


In [56]:
tokyo_data_dict

[{'district': 'Taito Ku',
  'date': '不明',
  'address': '東京都台東区橋場二丁目3-11',
  'content': 'メロンハイツ2階\n\n精神的瑕疵アリ'},
 {'district': 'Taito Ku',
  'date': '不明',
  'address': '東京都台東区橋場二丁目3-11',
  'content': 'メロンハイツ2階\n\n精神的瑕疵アリ'},
 {'district': 'Taito Ku',
  'date': '令和2年9月22日',
  'address': '東京都台東区橋場二丁目1-7橋場トドロキ6階',
  'content': '孤高死'},
 {'district': 'Taito Ku',
  'date': '2008年以前',
  'address': '東京都台東区花川戸一丁目13-6',
  'content': '共用部外階段より投身自殺'},
 {'district': 'Taito Ku',
  'date': '平成22年12月19日',
  'address': '東京都台東区花川戸一丁目15-11',
  'content': ''},
 {'district': 'Taito Ku', 'date': '', 'address': '', 'content': ''},
 {'district': 'Taito Ku', 'date': '', 'address': '', 'content': ''},
 {'district': 'Taito Ku', 'date': '', 'address': '', 'content': ''},
 {'district': 'Taito Ku',
  'date': '平成27年6月23日',
  'address': '東京都墨田区両国一丁目12-13',
  'content': '2015年6月23日18時ごろ\n飛び降り自殺 詳細不明'},
 {'district': 'Taito Ku',
  'date': '令和4年5月30日',
  'address': '東京都墨田区両国一丁目13-4 アイル東京リバーサイド',
  'content': '6階 角部屋 東向き1K（

In [57]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
tokyo_df

,district,date,address,content
0,Taito Ku,不明,東京都台東区橋場二丁目3-11,メロンハイツ2階\n\n精神的瑕疵アリ
1,Taito Ku,不明,東京都台東区橋場二丁目3-11,メロンハイツ2階\n\n精神的瑕疵アリ
2,Taito Ku,令和2年9月22日,東京都台東区橋場二丁目1-7橋場トドロキ6階,孤高死
3,Taito Ku,2008年以前,東京都台東区花川戸一丁目13-6,共用部外階段より投身自殺
4,Taito Ku,平成22年12月19日,東京都台東区花川戸一丁目15-11,
...,...,...,...,...
2966,Tama Shi,不明,東京都多摩市和田1261-19,百草団地19号棟202号室\n特別募集住宅
2967,Tama Shi,2020年1月現在,東京都多摩市和田1465 ドウエル中和田 2階,告知事項あり（不動産情報より）
2968,Tama Shi,2020年1月現在,東京都多摩市和田1467 フラット87 1階,告知事項あり（不動産情報より）
2969,Tama Shi,2020年4月,東京都多摩市和田1183,告知事項有り（不動産管理会社情報より）１F


In [60]:
# 결측치 제거 
df_cleaned = tokyo_df.dropna(how='any', axis=0)

for column in df_cleaned.columns:
    df_cleaned = df_cleaned[df_cleaned[column] != '']

# 중복데이터 제거 
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned

,district,date,address,content
0,Taito Ku,不明,東京都台東区橋場二丁目3-11,メロンハイツ2階\n\n精神的瑕疵アリ
2,Taito Ku,令和2年9月22日,東京都台東区橋場二丁目1-7橋場トドロキ6階,孤高死
3,Taito Ku,2008年以前,東京都台東区花川戸一丁目13-6,共用部外階段より投身自殺
8,Taito Ku,平成27年6月23日,東京都墨田区両国一丁目12-13,2015年6月23日18時ごろ\n飛び降り自殺 詳細不明
9,Taito Ku,令和4年5月30日,東京都墨田区両国一丁目13-4 アイル東京リバーサイド,6階 角部屋 東向き1K（洋室7.4帖、K） 専有面積25.61㎡ 分譲賃貸\n心理的瑕疵有...
...,...,...,...,...
2960,Tama Shi,不明,東京都多摩市和田1261-26,百草団地26号棟206号室\n特別募集住宅
2961,Tama Shi,不明,東京都多摩市和田1261-19,百草団地19号棟202号室\n特別募集住宅
2967,Tama Shi,2020年1月現在,東京都多摩市和田1465 ドウエル中和田 2階,告知事項あり（不動産情報より）
2968,Tama Shi,2020年1月現在,東京都多摩市和田1467 フラット87 1階,告知事項あり（不動産情報より）


In [62]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1637 entries, 0 to 1636
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  1637 non-null   object
 1   date      1637 non-null   object
 2   address   1637 non-null   object
 3   content   1637 non-null   object
dtypes: object(4)
memory usage: 51.3+ KB


In [63]:

# df_cleaned.to_excel('selenium_tokyo_data_geo(-Nerima).xlsx')
df_cleaned.to_csv('selenium_tokyo_data_geo(-Tama Shi).csv')

In [65]:
df1 = pd.read_csv('selenium_tokyo_data_geo(-Nerima).csv', index_col=0)
df2 = pd.read_csv('selenium_tokyo_data_geo(-Tama Shi).csv', index_col=0)

all_data = pd.concat([df1, df2], ignore_index=True)
all_data

,district,date,address,content
0,Fussa Shi,平成22年7月頃,東京都福生市熊川1628-1,独居老人が室内で孤高死
1,Fussa Shi,平成22年5月25日,東京都昭島市美堀町五丁目16-10,独身居住者が病死し、遺体で発見された
2,Fussa Shi,2006年3月末頃,東京都福生市大字熊川1655-15,孤高死、もしくは毒殺。(コップに酒が残っており、口や鼻からも嘔吐物が有った為。)\n遺体はう...
3,Fussa Shi,時期不明,東京都昭島市緑町五丁目14-20 グリーンタウン・ミキ 2階/205号室,心理的瑕疵あり
4,Fussa Shi,令和4年9月4日,東京都昭島市緑町五丁目14-20 グリーンタウン・ミキ,313号室\n告知事項あり
...,...,...,...,...
2463,Tama Shi,不明,東京都多摩市和田1261-26,百草団地26号棟206号室\n特別募集住宅
2464,Tama Shi,不明,東京都多摩市和田1261-19,百草団地19号棟202号室\n特別募集住宅
2465,Tama Shi,2020年1月現在,東京都多摩市和田1465 ドウエル中和田 2階,告知事項あり（不動産情報より）
2466,Tama Shi,2020年1月現在,東京都多摩市和田1467 フラット87 1階,告知事項あり（不動産情報より）


#### 기존 오시마 데이터랑 중복 확인 

In [66]:
oshima_yun = pd.read_csv('../../Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col=0)
oshima_ji = pd.read_csv('../../Oshimaland_data/jieun/oshimaland_dataset_final.csv')


In [67]:
# 결측치 제거 
oshima_yun = oshima_yun.dropna(how='any', axis=0)

for column in oshima_yun.columns:
    oshima_yun = oshima_yun[oshima_yun[column] != '']


# # 중복데이터 제거 
oshima_yun = oshima_yun.drop_duplicates()
oshima_yun = oshima_yun.reindex(columns=['district', 'address', 'content', 'date'])
oshima_yun.rename(columns={'content': 'info'}, inplace=True)
oshima_yun

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
2,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
4,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
6,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
8,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
298,羽村市,東京都羽村市緑ヶ丘五丁目1-50,駐車スペース横の部屋で住民男性が首吊り自殺,2005
299,武蔵村山市,東京都羽村市羽中一丁目3-37,告知事項,不明
300,昭島市,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,死体発見,平成24年
301,福生市,東京都昭島市田中町一丁目34-9,火災による死亡,令和2年9月7日


In [68]:
oshima_ji_rename = oshima_ji.reindex(columns=[ 'address', 'info', 'occurred_date'])
oshima_ji_rename.rename(columns={'occurred_date': 'date'}, inplace=True)
oshima_ji_rename = oshima_ji_rename.drop_duplicates()
oshima_ji_rename.reset_index(drop=True, inplace=True)
oshima_ji_rename.dropna(inplace=True)
oshima_ji_rename = oshima_ji_rename.reindex(columns = ['district', 'address', 'info', 'date'])
oshima_ji_rename['district'] = oshima_ji_rename['address'].apply(lambda x: next((district for district in jp_district_list if district in x), x))
oshima_ji_rename.reindex(columns=['district', 'address', 'info', 'occurred_date'])

oshima_ji_rename


,district,address,info,date
0,新宿区,東京都新宿区歌舞伎町一丁目202,819号室で女性死亡,平成30年5月23日
1,新宿区,東京都新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月18日
2,新宿区,東京都新宿区西新宿七丁目1813,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,1年以内
3,新宿区,東京都新宿区西新宿七丁目1713パレステュディオ新宿WEST,１４Ｆから女性が飛び降り自殺,令和元年9月23日
4,新宿区,東京都新宿区西新宿七丁目2231,事故死,2016
...,...,...,...,...
561,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
562,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
563,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
564,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [69]:
combined_df = pd.concat([oshima_yun, oshima_ji_rename], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
803,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
804,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
805,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
806,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [70]:
new_oshima = all_data.reindex(columns=['district', 'address', 'content', 'date'])
new_oshima.rename(columns={'content': 'info'}, inplace=True)
new_oshima

,district,address,info,date
0,Fussa Shi,東京都福生市熊川1628-1,独居老人が室内で孤高死,平成22年7月頃
1,Fussa Shi,東京都昭島市美堀町五丁目16-10,独身居住者が病死し、遺体で発見された,平成22年5月25日
2,Fussa Shi,東京都福生市大字熊川1655-15,孤高死、もしくは毒殺。(コップに酒が残っており、口や鼻からも嘔吐物が有った為。)\n遺体はう...,2006年3月末頃
3,Fussa Shi,東京都昭島市緑町五丁目14-20 グリーンタウン・ミキ 2階/205号室,心理的瑕疵あり,時期不明
4,Fussa Shi,東京都昭島市緑町五丁目14-20 グリーンタウン・ミキ,313号室\n告知事項あり,令和4年9月4日
...,...,...,...,...
2463,Tama Shi,東京都多摩市和田1261-26,百草団地26号棟206号室\n特別募集住宅,不明
2464,Tama Shi,東京都多摩市和田1261-19,百草団地19号棟202号室\n特別募集住宅,不明
2465,Tama Shi,東京都多摩市和田1465 ドウエル中和田 2階,告知事項あり（不動産情報より）,2020年1月現在
2466,Tama Shi,東京都多摩市和田1467 フラット87 1階,告知事項あり（不動産情報より）,2020年1月現在


In [71]:
combined_df2 = pd.concat([combined_df, new_oshima], ignore_index=True)
combined_df2 = combined_df2.drop_duplicates()
combined_df2

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
3266,Tama Shi,東京都多摩市和田1261-26,百草団地26号棟206号室\n特別募集住宅,不明
3267,Tama Shi,東京都多摩市和田1261-19,百草団地19号棟202号室\n特別募集住宅,不明
3268,Tama Shi,東京都多摩市和田1465 ドウエル中和田 2階,告知事項あり（不動産情報より）,2020年1月現在
3269,Tama Shi,東京都多摩市和田1467 フラット87 1階,告知事項あり（不動産情報より）,2020年1月現在


#### 중복 데이터 확인 
- 7개의 지역만 추가 트롤링 했을때 기존 807개의 데이터에서  1183개의 데이터 즉 376 개의 새로운 데이터 수집 
- 대략적으로 46개의 지역을 다 크롤링 할 시 2000개 이상의 데이터 추가 수집 가능 

In [72]:
# address 컬럼의 값이 중복되면 하나만 남기고 제거
fin_data = combined_df2.drop_duplicates(subset=['address'], keep='first')
fin_data

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
3258,Tama Shi,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃
3259,Tama Shi,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃
3261,Tama Shi,東京都多摩市和田2009-1,飛び降り自殺,2018年2月
3262,Tama Shi,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日


In [73]:
fin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2563 entries, 0 to 3263
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  2563 non-null   object
 1   address   2563 non-null   object
 2   info      2563 non-null   object
 3   date      2563 non-null   object
dtypes: object(4)
memory usage: 100.1+ KB


In [74]:
fin_data.to_csv('oshima_data_combine_fin2.csv')

In [80]:
df1 = pd.read_csv('oshima_data_combine_fin.csv', index_col=0)
df2 = pd.read_csv('oshima_data_combine_fin2.csv', index_col=0)

final_data = pd.concat([df1, df2], ignore_index=True)
final_data.to_csv('final_data.csv')

#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })